# MODELISATION

Pour répondre à notre problématique, nous implémentons deux modèles de machine learning : Random Forest et XGBoost.

Enfin, nous allons utiliser des modèles de machine learning pour prédire le genre d'une musique grâce aux variables dont nous disposons. Nous utilisons d'abord un modèle de type Random Forest, puis un modèle de type XGBoost, pour finalement comparer les deux.

Nous avons réalisé cette partie à partir des articles de Ilyes TALBI, Samir JEETO et Valentin DORE.

### Importation

In [23]:
#Pour la récupération des données
import pandas as pd

#Pour la visualisation
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Pour la modélisation
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score, zero_one_loss, classification_report
from sklearn.preprocessing import normalize, StandardScaler
from pprint import pprint

In [27]:
file_path = "/tlaflotte/genre_detector/spotify_tracks_cleaned.csv"

df = pd.read_csv("https://minio.lab.sspcloud.fr" + file_path)

# I - Random Forest

Tout d'abord, nous séparons la variable à prédire (le genre) des autres variables (les features). Ensuite, nous séparons ces ensembles en deux parties, l'une pour entraîner le modèle et l'autre pour tester le modèle. Comme notre base de données est ordonnée par genre, nous indiquons qu'il faut bisséquer aléatoirement cette base de données avec l'argument shuffle=True.

Nous pourrions enlever des variables puisque notre étape de visualisation a mis en évidence de fortes corrélations entre certaines d'entre elles. Cela ferait gagner du temps, pour un petit peu moins de précision. Nous avons choisi de garder toutes nos variables car le temps de calcul n'est pas très long.

In [28]:
features = df.copy()

# isolation of the feature to predict
genres = np.array(features['genre'])
features = features.drop('genre', axis = 1)
feature_list = list(features.columns)
features = np.array(features)

# separation in training and testing sets
train_features, test_features, train_genres, test_genres = train_test_split(features, genres, test_size = 0.25, random_state = 0, shuffle = True)
print('Training Features Shape:', train_features.shape)
print('Training Genres Shape:', train_genres.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Genres Shape:', test_genres.shape)

# reshaping
sc = StandardScaler()
train_features = sc.fit_transform(train_features)
test_features = sc.transform(test_features)

KeyError: 'genre'

## Tuning des hyper-paramètres

Maintentant, nous allons procéder au tuning des hyper-paramètres.

In [20]:
rf = RandomForestClassifier(random_state = 0)

print('Parameters currently in use:\n')
print(rf.get_params())

Parameters currently in use:

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


Il y a bien trop d'hyper-paramètres sur lesquels nous pouvons jouer. Nous choisissons de jouer sur 8 d'entre eux tels que le nombre d'arbres, la profondeur maximale des arbres ou encore le nombre minimal d'échantillons par nœuds.

Nous n'avons pas indiqué des poids pour chaque genre, car la base de données est relativement équilibrée. Pour améliorer le modèle, nous pourrions mettre des poids à chaque genre pour avoir une base de données parfaitement équilibrée.

Voici, les valeurs pour lesquelles nous allons tester les performances du modèle.

In [24]:
# Créer un classifieur Random Forest
rf = RandomForestClassifier(random_state=0)

# Définir l'espace de recherche pour chaque hyperparamètre
param_dist = {
    'n_estimators': np.random.randint(100, 1000),  # entre 100 et 1000 arbres
    'max_depth': [None, 10, 20, 30, 40, 50],  # Profondeur des arbres
    'min_samples_split': np.random.randint(2, 20),  # Entre 2 et 20 échantillons pour diviser un noeud
    'min_samples_leaf': np.random.randint(1, 20),  # Entre 1 et 20 échantillons par feuille
    'max_features': ['auto', 'sqrt', 'log2', None],  # Nombre de caractéristiques à essayer
    'bootstrap': [True, False],  # Utiliser le bootstrap ou non
    'ccp_alpha': np.random.uniform(0.0, 0.1),  # Paramètre pour la coupe des arbres (complexité)
    'max_samples': [None, 0.8, 0.9],  # Fraction d'échantillons à utiliser pour chaque arbre
}

Ensuite, nous utilisons `RandomizedSearchCV` pour rechercher dans l'espace de recherche pour chaque hyperparamètre de manière aléatoire. Il y a un paramètre clé ici : le nombre d'itérations (nombre de combinaisons d'hyperparamètres à tester). Plus le nombre d'itérations est élevé, plus la recherche a de chances de trouver une bonne combinaison, mais cela prend plus de temps.

In [ ]:
# Définir la recherche aléatoire avec un nombre d'itérations (par exemple, 100)
random_search = RandomizedSearchCV(
    estimator=rf,  # Le modèle à entraîner
    param_distributions=param_dist,  # L'espace de recherche
    n_iter=100,  # Nombre d'itérations de la recherche
    scoring='accuracy',  # Critère d'évaluation (par exemple, l'exactitude)
    cv=5,  # Validation croisée (par exemple, 5-fold)
    verbose=1,  # Afficher les informations pendant l'exécution
    n_jobs=-1,  # Utiliser tous les cœurs du processeur pour accélérer le calcul
    random_state=0  # Garantir la reproductibilité des résultats
)

# Lancer la recherche
random_search.fit(train_features, train_genre)

# Résultats des meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres :", random_search.best_params_)
print("Meilleure performance (score) :", random_search.best_score_)

NameError: name 'RandomForestClassifier' is not defined

Une fois la recherche terminée, on peut accéder aux meilleurs hyperparamètres trouvés et à la performance correspondante via `random_search.best_params_` et `random_search.best_score_` qui renvoient respectivement les meilleurs hyperparamètres trouvés par la recherche et la performance (ici l'accuracy) du modèle avec ces meilleurs hyperparamètres.

Nous avons donc fini la partie du tuning des hyper-paramètres, et nous pouvons alors lancer le modèle.

## Lancement du modèle

In [ ]:
# Créer un nouveau classifieur Random Forest avec les meilleurs hyperparamètres
optimized_rf = RandomForestClassifier(**best_params, random_state=0)

# Entraîner le modèle sur les données d'entraînement
optimized_rf.fit(train_features, train_genre)

# prédictions
predictions = optimized_rf.predict(test_features)

# Zero_one_loss error
errors = zero_one_loss(test_genres, predictions, normalize=True)
print('zero_one_loss error normalized:', errors)

# Accuracy Score
accuracy_test = accuracy_score(test_genres, predictions)
print('accuracy_score on test dataset :', accuracy_test)

print(classification_report(predictions, test_genres))

NameError: name 'RandomForestClassifier' is not defined

Nous obtenons une précision de 65% ce qui est satisfaisant !

On remarque que les genres les mieux prédits sont bien classical music et rap, comme nous l'avions attendu grâce à notre étape de visualisation. De plus, les genres rock, r&b et encore pop sont moins bien identifés comme nous l'attendions également.

Nous traçons la matrice de confusion pour visualiser les différentes erreurs.

In [ ]:
#Confusion matrix

mat = confusion_matrix(test_genres, predictions)
plt.imshow(mat, cmap='viridis', interpolation='nearest')
#plt.colorbar(label='Valeurs')
plt.title('Matrice de Confusion')
num_rows, num_cols = mat.shape
for i in range(num_rows):
    for j in range(num_cols):
        plt.text(j, i, str(mat[i, j]), ha='center', va='center', color='w', fontsize=12)
plt.grid(False)
genres_list = ['edm','latin','pop','r&b','rap','rock']
plt.xticks(np.arange(num_cols), genres_list, rotation=45, ha='right')
plt.yticks(np.arange(num_rows), genres_list)
plt.xlabel('true genre')
plt.ylabel('predicted genre')
plt.show()

NameError: name 'confusion_matrix' is not defined

Grâce à la matrice des confusions, nous pouvons voir par exemple que les musiques classiques sont bien classés, sauf quelques unes qui sont évaluées comme des musiques jazz : c'est exactement ce que notre étape de visualisation avait annoncé !

In [8]:
#Feature importances
plt.style.use('fivethirtyeight')

importances = list(rf.feature_importances_)

x_values = list(range(len(importances)))
plt.bar(x_values, importances, orientation = 'vertical')
plt.xticks(x_values, feature_list, rotation='vertical')
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances')
plt.show()

NameError: name 'plt' is not defined

Enfin, nous traçons ici l'importance de chaque variable. Nous observons que key, mode et time_signature ne sont pas très utiles à la prédiction. Nous pouvions nous y attendre puisque ce sont des variables qui varient relativement peu, et qui sont décorrélées de chaque genre.

Ainsi, notre modélisation Random Forest permet de prédire un genre musical avec une précision de 66%, ce qui est satisfaisant, mais en réalité elle identifie mal les genres pop et rock qui sont très représentés dans le monde de la musique, tandis qu'elle identifie bien le genre classical music, qui n'est pas le genre le plus écouté de nos jours.